# Calculate RMSD for evaluating performance of DiffDock runs

### Import libraries

In [2]:
from spyrmsd import io, rmsd
import os
import re
import numpy as np

In [3]:
def calculateRMSD(pdb_name, diffdock_filename, results_type):
    """Calculation of RMSD between actual and predicted ligand poses"""
    # convert predicted and actual ligand
    ref = io.loadmol("ligand_sdf/"+pdb_name+"_ligand.sdf")
    mol = io.loadmol("./DiffDock_results/"+results_type+"_results/"+pdb_name+"/"+diffdock_filename)

    # remove hydrogens
    ref.strip()
    mol.strip()

    # extract atomic coordinates, atomic numbers and the molecular adjacency matrix
    coords_ref = ref.coordinates
    anum_ref = ref.atomicnums
    adj_ref = ref.adjacency_matrix
    coords = mol.coordinates
    anum = mol.atomicnums
    adj = mol.adjacency_matrix

    # calculate RMSD
    RMSD = rmsd.symmrmsd(
    coords_ref,
    coords,
    anum_ref,
    anum,
    adj_ref,
    adj,
    )

    return RMSD

def StoreRMSDInfo(results_type, ranks):
    """Creates output file for storing RMSDs for the different runs"""
    # extract pdb names
    pdb_dir = os.listdir("./DiffDock_results/"+results_type+"_results")
    pdb_names = [item for item in pdb_dir if len(item) == 4]

    # initialize
    conf_dict = {list_: [] for list_ in range(ranks)}


    # write information to outfile
    confidence_rmsd_outfile = open("./DiffDock_RMSD/information_rmsd"+results_type+".tab", "w")
    confidence_rmsd_outfile.write("rank\trmsd_"+ results_type +"\tconfidence_score"+ results_type +"\tpdb_name\n")

    for i in range(len(pdb_names)):
        try:
            pred_ligands_filenames = os.listdir("./DiffDock_results/"+results_type+"_results/"+pdb_names[i])
            pred_ligands_filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            pred_ligands_filenames = pred_ligands_filenames[1:]
            assert(len(pred_ligands_filenames) == ranks)
            for j in range(len(pred_ligands_filenames)):
                # get correct ranks at every time (based on confidence)
                rank = re.findall(r'\d+', pred_ligands_filenames[j])[0]

                # extract confidence score
                assert(len(re.findall(r'\d+.\d+', pred_ligands_filenames[j])) == 1)
                confidence_score = re.findall(r'\d+.\d+', pred_ligands_filenames[j])[0]

                # get RMSD grouped on rank
                assert(int(rank) == j+1)
                RMSD = calculateRMSD(pdb_name = pdb_names[i], 
                    diffdock_filename = pred_ligands_filenames[j],
                    results_type = results_type)
                conf_dict[j].append(RMSD)
                # write rank, RMSD, confidence score and pdb name of reference protein to outfile
                confidence_rmsd_outfile.write(str(j+1) + "\t" + str(RMSD) +"\t" + confidence_score + "\t" + pdb_names[i] + "\n")

        except OSError as err:
            print(str(err))
    confidence_rmsd_outfile.close()


In [ ]:
# get results for several runs
StoreRMSDInfo("DiffDock10_long", 10)
StoreRMSDInfo("DiffDock10", 10)
StoreRMSDInfo("DiffDock40_long", 40)
StoreRMSDInfo("DiffDock40", 40)